# Functional API


Not all neural network models are simply sequential. Some may have complex topologies. Some may have multiple inputs and/or multiple outputs. For example, a Wide & Deep neural network (see paper) connects all or part of the inputs directly to the output layer.
Let’s build such a neural network to tackle the California housing problem:

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [3]:
np.random.seed(42)
tf.random.set_seed(42)

In [4]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)



In [5]:
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.models.Model(inputs=[input_], outputs=[output])


In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 30)           270         ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 30)           930         ['dense[0][0]']                  
                                                                                                  
 concatenate (Concatenate)      (None, 38)           0           ['input_1[0][0]',                
                                                                  'dense_1[0][0]']            

In [8]:
model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3]
y_pred = model.predict(X_new)

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 0.4492 - val_loss: 0.4691
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4465 - val_loss: 0.4643
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4435 - val_loss: 0.4630
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4411 - val_loss: 0.4605
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4395 - val_loss: 0.4579
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4363 - val_loss: 0.4570
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4338 - val_loss: 0.4529
Epoch 8/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4313 - val_loss: 0.4513
Epoch 9/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4291 - val_loss: 0.4496
Epoch 10/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4274 - val_loss: 0.4477

What if you want to send different subsets of input features through the wide or deep paths? We will send 5 features (features 0 to 4), and 6 through the deep path (features 2 to 7). Note that 3 features will go through both (features 2, 3 and 4).


In [9]:
np.random.seed(42)
tf.random.set_seed(42)

In [10]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.models.Model(inputs=[input_A, input_B], outputs=[output])

In [11]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

In [12]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [13]:
history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))
mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 2.0909 - val_loss: 0.8706
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 0.7442 - val_loss: 0.6824
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.6510 - val_loss: 0.6258
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5936 - val_loss: 0.5928
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5703 - val_loss: 0.5698
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5447 - val_loss: 0.5554
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5298 - val_loss: 0.5427
Epoch 8/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5180 - val_loss: 0.5340
Epoch 9/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5097 - val_loss: 0.5267
Epoch 10/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5026 - val_loss: 0.5208

Adding an auxiliary output for regularization:

In [22]:
np.random.seed(42)
tf.random.set_seed(42)

In [23]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
model = keras.models.Model(inputs=[input_A, input_B],
                           outputs=[output, aux_output])

In [24]:
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(lr=1e-3))

In [25]:
history = model.fit([X_train_A, X_train_B], [y_train, y_train], epochs=20, validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 2s 132us/sample - loss: 2.1388 - main_output_loss: 1.9341 - aux_output_loss: 3.9759 - val_loss: 1.0634 - val_main_output_loss: 0.8326 - val_aux_output_loss: 3.1391
Epoch 2/20
11610/11610 [==============================] - 1s 86us/sample - loss: 0.9184 - main_output_loss: 0.7344 - aux_output_loss: 2.5738 - val_loss: 0.8119 - val_main_output_loss: 0.6546 - val_aux_output_loss: 2.2267
Epoch 3/20
11610/11610 [==============================] - 1s 92us/sample - loss: 0.7613 - main_output_loss: 0.6333 - aux_output_loss: 1.9130 - val_loss: 0.7349 - val_main_output_loss: 0.6129 - val_aux_output_loss: 1.8316
Epoch 4/20
11610/11610 [==============================] - 1s 66us/sample - loss: 0.7012 - main_output_loss: 0.5986 - aux_output_loss: 1.6236 - val_loss: 0.6949 - val_main_output_loss: 0.5897 - val_aux_output_loss: 1.6402
Epoch 5/20
11610/11610 [==============================] - 1s 56us/

In [26]:
total_loss, main_loss, aux_loss = model.evaluate(
    [X_test_A, X_test_B], [y_test, y_test])
y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])

5160/5160 [==============================] - 0s 29us/sample - loss: 0.5300 - main_output_loss: 0.4847 - aux_output_loss: 0.9471
